# Ground Truth でカスタムのラベリングタスクを実施する

このノートブックでは，Ground Truth を使った簡単なカスタムタスクを実際に試してみます．その際に必要な Lambda や jekyll テンプレートについても確認します．このノートブックを実施する前に，すでに `simple_labelling_task` のノートブックを実行している必要があります．

## セットアップ

### Lambda 関数の作成

前準備として，まず Lambda 関数を 2 つ作成します．これは，ラベリングジョブの前処理とあと処理を行うためのものです．それぞれの役割は以下の通りです．

- 前処理: manifest.json から必要な内容を読み込んで，jekyll テンプレートに引き渡す役割
- 後処理: ラベリングされた各ワーカーの結果を集約する役割

In [ ]:
!cat src/ground-truth-preprocess.py

In [ ]:
!cat src/ground-truth-postprocess.py

[Lambda のマネジメントコンソール](https://console.aws.amazon.com/lambda/home?region=us-east-1)を開いて，それぞれ以下の形で作成する．Name, Runtime, Role を設定したら `Create function` を選択して，次のページで Function code の中身をコピーする形になる．その上で，右上の `Save` ボタンを押す．

#### 前処理

- Name: ground-truth-preprocess
- Runtime: Python 3.7
- Role: arn:aws:iam::aws:policy/AWSLambdaExecute ポリシーを付与した Role
- Function code: ground-truth-preprocess.py の内容をコピーして貼り付け

#### 後処理

- Name: ground-truth-postprocess
- Runtime: Python 2.7
- Role: `Create a new role from one or more templetes` を選択して，`Amazon S3 object read-only permissions` を選ぶ．ロール名は適当なものをつける
- Function code: ground-truth-preprocess.py の内容をコピーして貼り付け

### テンプレートの確認

以下のコマンドを実行して，カスタムタスク用のテンプレートの中身を確認します．ここでは，Ground Truth で定義されている要素を活用して，Jekyll 向けのテンプレートを構成しています．定義されている HTML 要素の一覧については，[こちら](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-reference.html)を参照してください．機械学習に関するものとしては，例えば以下のような特定タスク向けのタグがあります．これらのタグは，crowd-form の子要素となっているため，入力結果を書き出すことができます．書き出した結果は S3 に保存されるため，S3 キー名を後処理の Lambda で受け取って，ファイル自体を読み込んで複数ワーカーの結果をマージする形になります．

- [crowd-classifier](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-classifier.html)
- [crowd-image-classifier](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-image-classifier.html)
- [crowd-bounding-box](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-bounding-box.html)
- [crowd-semantic-segmentation](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-semantic-segmentation.html)

またそれ以外にも，下記のようなタグも入力部品としてお使いいただくことが可能です．こちらの入力結果も，同様に後処理の Lambda に引き渡されます．

- [crowd-input](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-input.html)
- [crowd-slider](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-slider.html)
- [crowd-button](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-ui-template-crowd-button.html)


このテンプレートの中で書かれている，task.input.sourceRef といったパラメタは，前処理用の Lambda から引き渡されるものになります．また後処理の Lambda の処理とも合わせて，このあたりの詳細については，[マニュアル](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-custom-templates-step2-demo1.html)を参照してください．


In [ ]:
!cat src/custom-task.templete

### パラメタの設定

以下のパラメタを設定します．内容は simple_labelling_task のノートブックで指定したのと同じものにしてください．

In [ ]:
import sagemaker

BUCKET_NAME = sagemaker_session.default_bucket()
PATH = 'data/animal'
JOB_NAME = 'custom-job-XXX'

## ラベリングジョブの作成

まず，以下のコマンドを実行して，必要な値を生成します．

In [ ]:
print('Job name: {}'.format(JOB_NAME))
print('Input data location: s3://{}/{}/{}/manifest.json'.format(
    sagemaker_session.default_bucket(), JOB_NAME, PATH))
print('Output data location: s3://{}'.format(sagemaker_session.default_bucket()))

[SageMaker コンソールのラベリングジョブページ](https://console.aws.amazon.com/sagemaker/groundtruth?region=us-east-1#/labeling-jobs) に移動して，`Create labelling job ` ボタンをクリックします．その後以下のように入力を行ってから，「Next」ボタンを押してください．

- Job name: 上の値をそのままコピー
- Input dataset location: 上の値をそのままコピー
- Output dataset location: 上の値をそのままコピー
- IAM role: 既存の SageMakerFullAccess ポリシーを付与したロールを使用

その上で，Task type として Custom を選択したら Next を押してください．

次のページでは，順番に以下のように選択してください．

- Worker types: Private を選択
- Private teams: 先ほど作成したチームを選択

その上で，Custom labeling task setup について，以下のように追加情報の入力を行います．

- Templete: Custom
- Templete code: src/custom-task.templete の中身をそのままコピペする

また一番下の Lambda 関数を指定する箇所は，以下のようにします．

- Pre-labeling task Lambda function: ground-truth-preprocess
- Post-labeling task Lambda function: ground-truth-postprocess

これらが終わったら Submit ボタンを押してください．

![labelling_image](./img/img001.jpg)

## ラベリングジョブの実施

あとは，先ほどの Worker 側作業ページを開いて待っていると，分類タスクが積まれます．実際にラベリングタスクを試してみてください．プリセットの物体検出テンプレートを使った場合だと，1 ジョブでは 1 つのクラスの物体しか検出することができませんでしたが，カスタムテンプレートを使うことで，eye, nose の 2 つの物体を検出対象にできるようになっています．

![labelling_image](./img/img002.jpg)


## 結果の確認

ジョブが終了したら，マネジメントコンソール内のラベリングジョブの一覧から，実行したジョブを選択して，Output dataset location をクリックしてください．アウトプットフォルダは，以下のような階層構造になっています．

- s3://PATH/TO/JOB/OUTPUT/activelearning: 自動ラベリング機能を使った時に，その結果を格納するディレクトリです
- s3://PATH/TO/JOB/OUTPUT/annotations: 人手によるラベリングジョブの結果が格納されるディレクトリです．個々人のラベリング結果を入れる worker-responce サブディレクトリ，各バッチごとのアノテーション結果をまとめた consolidated-annotation サブディレクトリ，バッチ内の判定結果をマニフェストファイルの形に落とした intermediate サブディレクトリの 3 つが含まれます
- s3://PATH/TO/JOB/OUTPUT/inference: 自動ラベリング機能の実施時の，バッチ推論ジョブの入出力データが格納されるディレクトリです
- s3://PATH/TO/JOB/OUTPUT/manifests: 最終結果のマニフェストファイルが格納されるディレクトリです
- s3://PATH/TO/JOB/OUTPUT/training: 自動ラベリング機能の実施時の，学習ジョブに関するデータが格納されるディレクトリです

上記以外の詳細な説明は[こちら](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-data-output.html)をご覧ください．

では，最終的なラベリング結果を取得して中身を確認しましょう．下記コマンドを実行して，中身を確認します．

In [ ]:
import sys

s3.download_file(BUCKET_NAME, '{}/manifests/output/output.manifest'.format(JOB_NAME), 'output.manifest')

with open('output.manifest', 'r') as f:
    for line in f.readlines():
        print(line)